In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

Randomized search

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler

In [3]:
# import data
df= pd.read_table('datas.txt')
df.head()

,WD,WFS,TS,P,ME
0,0.8,39.33,5.00,531.22,0.396
1,0.8,48.33,6.66,693.84,0.414
2,0.8,66.83,8.34,959.70,0.433
3,0.8,80.00,10.00,1113.30,0.446
4,0.8,94.33,11.67,1277.70,0.445


In [4]:
df.columns

Index(['WD', 'WFS', 'TS', 'P', 'ME'], dtype='object')

In [5]:
# Split data in x and y
x= df[['WD', 'WFS', 'TS', 'P']]
y= df[['ME']]

In [6]:
# Split data into train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(60, 4) (15, 4) (60, 1) (15, 1)


In [7]:
# Feature scaling
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.fit_transform(x_test)

## GP

In [8]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

gp= GaussianProcessRegressor(alpha=0.001, copy_X_train=True,
                         kernel=1**2 * RBF(length_scale=10),
                         n_restarts_optimizer=40, normalize_y=True,
                         optimizer='fmin_l_bfgs_b', random_state=1234)
gp.fit(x_train, y_train)

GaussianProcessRegressor(alpha=0.001, kernel=1**2 * RBF(length_scale=10),
                         n_restarts_optimizer=40, normalize_y=True,
                         random_state=1234)

In [9]:
y_train_gp= gp.predict(x_train)
y_test_gp= gp.predict(x_test)

In [10]:
# Mean squared erro
from sklearn.metrics import mean_squared_error
mse_train_gp= mean_squared_error(y_train, y_train_gp)
mse_test_gp= mean_squared_error(y_test, y_test_gp)
print(mse_train_gp)
print('............')
print(mse_test_gp)

8.063344356780536e-08
............
0.0008340061786338994


In [11]:
# R2 score
from sklearn.metrics import r2_score
r2_train_gp= r2_score(y_train, y_train_gp)
r2_test_gp= r2_score(y_test, y_test_gp)
print(r2_train_gp)
print('............')
print(r2_test_gp)

0.9999727852262603
............
0.7150433085517849


In [12]:
# Function defination for R2 and Adj R2

def r2(actual: np.ndarray, predicted: np.ndarray):
    """ R2 Score """
    return r2_score(actual, predicted)

def adjr2(actual: np.ndarray, predicted: np.ndarray, rowcount: np.int, featurecount: np.int):
    """ Adj R2 Score """
    return 1-(1-r2(actual,predicted))*(rowcount-1)/(rowcount-featurecount)

In [13]:
r2_train_gp= r2(y_train, y_train_gp)
r2_test_gp= r2(y_test, y_test_gp)

adj_r2_train_gp= adjr2(y_train, y_train_gp, 60, 2)
adj_r2_test_gp= adjr2(y_test, y_test_gp, 15, 2)

print(r2_train_gp)
print('............')
print(r2_test_gp)
print('............')
print(adj_r2_train_gp)
print('............')
print(adj_r2_test_gp)

0.9999727852262603
............
0.7150433085517849
............
0.9999723160060234
............
0.6931235630557684


In [14]:
# Mean absolute error
from sklearn.metrics import mean_absolute_error
mae_train_gp= mean_absolute_error(y_train, y_train_gp)
mae_test_gp= mean_absolute_error(y_test, y_test_gp)
print(mae_train_gp)
print('............')
print(mae_test_gp)

0.00018662792820824854
............
0.022455431820443163


In [15]:
# RMSE
rmse_train_gp = np.sqrt(mse_train_gp)
rmse_test_gp = np.sqrt(mse_test_gp)
print(rmse_train_gp)
print('............')
print(rmse_test_gp)

0.00028396028519461194
............
0.028879165130486363


In [16]:
#Randomized search
# initialize model and define the space of the hyperparameters to
# perform the grid-search over
model = gp
n_restarts_optimizer = [5, 10, 20, 40]
alpha = [0.001, 0.01, 0.1, 1]


grid = dict(n_restarts_optimizer=n_restarts_optimizer, alpha=alpha)

In [17]:
# initialize a cross-validation fold and perform a randomized-search
# to tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1234)
randomSearch = RandomizedSearchCV(estimator=model, n_jobs=-1,
	cv=cvFold, param_distributions=grid,
	scoring="neg_mean_squared_error")
searchResults = randomSearch.fit(x_train, y_train)
# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
print("R2: {:.2f}".format(bestModel.score(x_test, y_test)))
print(bestModel)

[INFO] grid searching over the hyperparameters...
[INFO] evaluating...
R2: 0.85
GaussianProcessRegressor(alpha=0.1, kernel=1**2 * RBF(length_scale=10),
                         n_restarts_optimizer=10, normalize_y=True,
                         random_state=1234)


### Evaluation metrics

|Sr.No.|Adj R2 | R2 | MSE | RMSE | MAE |
|---|---| --- | --- | --- | --- |
|Train|0.99| 0.99 | 8.06e-8 | 0.0002 | 0.0001 |
|Test| 0.69 |0.71 | 8.3e-4 | 0.0288 | 0.022 |

## XGB

In [18]:
from xgboost import XGBRegressor
xgb= XGBRegressor()
xgb.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [19]:
y_train_xgb= xgb.predict(x_train)
y_test_xgb= xgb.predict(x_test)

In [20]:
r2_train_xgb= r2(y_train, y_train_xgb)
r2_test_xgb= r2(y_test, y_test_xgb)

adj_r2_train_xgb= adjr2(y_train, y_train_xgb, 60, 2)
adj_r2_test_xgb= adjr2(y_test, y_test_xgb, 15, 2)

print(r2_train_xgb)
print('............')
print(r2_test_xgb)
print('............')
print(adj_r2_train_xgb)
print('............')
print(adj_r2_test_xgb)

0.9998111730968064
............
0.7878886874042836
............
0.9998079174605445
............
0.7715724325892286


In [21]:
# Mean squared error
from sklearn.metrics import mean_squared_error
mse_train_xgb= mean_squared_error(y_train, y_train_xgb)
mse_test_xgb= mean_squared_error(y_test, y_test_xgb)
print(mse_train_xgb)
print('............')
print(mse_test_xgb)

5.594668391655011e-07
............
0.0006208036188373633


In [22]:
# Mean absolute error
from sklearn.metrics import mean_absolute_error
mae_train_xgb= mean_absolute_error(y_train, y_train_xgb)
mae_test_xgb= mean_absolute_error(y_test, y_test_xgb)
print(mae_train_xgb)
print('............')
print(mae_test_xgb)

0.0006044409791628568
............
0.019862835073471072


In [23]:
# RMSE
rmse_train_xgb = np.sqrt(mse_train_xgb)
rmse_test_xgb = np.sqrt(mse_test_xgb)
print(rmse_train_xgb)
print('............')
print(rmse_test_xgb)

0.0007479751594575191
............
0.024915931024895765


In [24]:
# initialize model and define the space of the hyperparameters to
# perform the grid-search over
model = xgb
n_estimators = [10, 100, 1000, 10000]
learning_rate = [0.1, 0.01, 1e-3]
max_depth = [5, 10, 20, 40]
reg_lambda = [0.01, 0.1, 1]


grid = dict(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth )

In [25]:
# initialize a cross-validation fold and perform a randomized-search
# to tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
randomSearch = RandomizedSearchCV(estimator=model, n_jobs=-1,
	cv=cvFold, param_distributions=grid,
	scoring="neg_mean_squared_error")
searchResults = randomSearch.fit(x_train, y_train)
# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
#print("R2: {:.2f}".format(bestModel.score(x_test, y_test)))
print(bestModel)

[INFO] grid searching over the hyperparameters...
[INFO] evaluating...
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)


### Evaluation metrics

|Sr.No.|Adj R2 | R2 | MSE | RMSE | MAE |
|---|---| --- | --- | --- | --- |
|Train|0.99| 0.99 | 5.59e-7 | 0.0007 | 0.0006 |
|Test| 0.77 |0.78 | 6.2e-4 | 0.0249 | 0.019 |

In [26]:
# Feature importance analysis

importance = xgb.feature_importances_
# summarize feature importance

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.02669
Feature: 1, Score: 0.10854
Feature: 2, Score: 0.09959
Feature: 3, Score: 0.76517


## ANN

In [27]:
from sklearn.neural_network import MLPRegressor

mlp= MLPRegressor(activation='tanh', hidden_layer_sizes=8, solver='lbfgs')
mlp.fit(x_train, y_train)

MLPRegressor(activation='tanh', hidden_layer_sizes=8, solver='lbfgs')

In [28]:
y_train_mlp= mlp.predict(x_train)
y_test_mlp= mlp.predict(x_test)

In [29]:
r2_train_mlp= r2(y_train, y_train_mlp)
r2_test_mlp= r2(y_test, y_test_mlp)

adj_r2_train_mlp= adjr2(y_train, y_train_mlp, 60, 2)
adj_r2_test_mlp= adjr2(y_test, y_test_mlp, 15, 2)

print(r2_train_mlp)
print('............')
print(r2_test_mlp)
print('............')
print(adj_r2_train_mlp)
print('............')
print(adj_r2_test_mlp)

0.9587534257523816
............
0.4512365048911887
............
0.9580422779205262
............
0.4090239283443571


In [30]:
# Mean squared error
from sklearn.metrics import mean_squared_error
mse_train_mlp= mean_squared_error(y_train, y_train_mlp)
mse_test_mlp= mean_squared_error(y_test, y_test_mlp)
print(mse_train_mlp)
print('............')
print(mse_test_mlp)

0.00012220764165719089
............
0.0016061112416890005


In [31]:
# RMSE
rmse_train_mlp = np.sqrt(mse_train_mlp)
rmse_test_mlp = np.sqrt(mse_test_mlp)
print(rmse_train_mlp)
print('............')
print(rmse_test_mlp)

0.011054756517318276
............
0.04007631771618995


In [32]:
# Mean absolute error
from sklearn.metrics import mean_absolute_error
mae_train_mlp= mean_absolute_error(y_train, y_train_mlp)
mae_test_mlp= mean_absolute_error(y_test, y_test_mlp)
print(mae_train_mlp)
print('............')
print(mae_test_mlp)

0.008750635292263902
............
0.03078545807203265


In [33]:
# initialize model and define the space of the hyperparameters to
# perform the grid-search over
model = mlp
hidden_layer_sizes = [5, 8, 10]
activation = ['relu','tanh']
solver = ["lbfgs", "sgd", "adam"]

grid = dict(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver )

In [34]:
# initialize a cross-validation fold and perform a randomized-search
# to tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1234)
randomSearch = RandomizedSearchCV(estimator=model, n_jobs=-1,
	cv=cvFold, param_distributions=grid,
	scoring="neg_mean_squared_error")
searchResults = randomSearch.fit(x_train, y_train)
# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
print("R2: {:.2f}".format(bestModel.score(x_test, y_test)))
print(bestModel)

[INFO] grid searching over the hyperparameters...
[INFO] evaluating...
R2: 0.53
MLPRegressor(activation='tanh', hidden_layer_sizes=8, solver='lbfgs')


### Evaluation metrics

|Sr.No.|Adj R2 | R2 | MSE | RMSE | MAE |
|---|---| --- | --- | --- | --- |
|Train|0.94| 0.95 | 0.0001 | 0.012 | 0.009 |
|Test| 0.57 |0.60 | 0.001 | 0.033 | 0.027 |

In [35]:
## Cross validation
from sklearn.model_selection import cross_val_score as CV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

models = [ xgb,  gp, mlp, ]
labels = ['XGBR', 'GPs', 'MLP', '']
IMS = []

print("--------------------------------------------------------------")
print("%5s | %5s | %5s | %5s | %5s " % ("ML algorithm",  "CV-R2", "CV-MSE", "CV-MAE", "IM"))
print("--------------------------------------------------------------")

for mod, label in zip( models, labels): 
  y_star_train = mod.predict(x_train)
 
  scoreR2 = CV(mod, x_train, y_train, cv=5, scoring='r2')
  #print(-1*scoreR2)
  scoreMSE = CV(mod, x_train, y_train, cv=5, scoring='neg_mean_squared_error')
  #print(-1*scoreMSE)
  scoreMAE = CV(mod, x_train, y_train, cv=5, scoring='neg_mean_absolute_error')
  #print(-1*scoreMAE)
  IM=np.sqrt(((1-scoreR2)**2) + (scoreMSE) + (scoreMAE**2) )
  #print("IM: ", IM)
  
 
  print("%15s | %.4f | %.4f | %.4f | %.4f " % (label,  scoreR2.mean(), -1*scoreMSE.mean(), -1*scoreMAE.mean() , np.sqrt(((1-scoreR2.mean())**2) + (-1*scoreMSE.mean()) + (scoreMAE.mean()**2) )))
print("--------------------------------------------------------------")

--------------------------------------------------------------
ML algorithm | CV-R2 | CV-MSE | CV-MAE |    IM 
--------------------------------------------------------------
           XGBR | 0.8611 | 0.0004 | 0.0145 | 0.1409 
            GPs | 0.6685 | 0.0009 | 0.0225 | 0.3336 
            MLP | 0.8347 | 0.0004 | 0.0159 | 0.1673 
--------------------------------------------------------------


In [39]:
x

,WD,WFS,TS,P
0,0.8,39.33,5.00,531.22
1,0.8,48.33,6.66,693.84
2,0.8,66.83,8.34,959.70
3,0.8,80.00,10.00,1113.30
4,0.8,94.33,11.67,1277.70
...,...,...,...,...
70,1.2,44.33,3.34,1688.92
71,1.2,57.33,4.17,1996.29
72,1.2,70.00,5.01,2177.65
73,1.2,80.17,5.83,2528.29


In [36]:
# Prediction of ME

y_ME_gp= gp.predict(x)
df['ME GP predicted']=y_ME_gp

y_ME_xgb= xgb.predict(x)
df['ME XGB predicted']=y_ME_xgb

y_ME_mlp= gp.predict(x)
df['ME MLP predicted']=y_ME_mlp

df

,WD,WFS,TS,P,ME,ME GP predicted,ME XGB predicted,ME MLP predicted
0,0.8,39.33,5.00,531.22,0.396,0.449333,0.531555,0.449333
1,0.8,48.33,6.66,693.84,0.414,0.449333,0.531555,0.449333
2,0.8,66.83,8.34,959.70,0.433,0.449333,0.531555,0.449333
3,0.8,80.00,10.00,1113.30,0.446,0.449333,0.531555,0.449333
4,0.8,94.33,11.67,1277.70,0.445,0.449333,0.531555,0.449333
...,...,...,...,...,...,...,...,...
70,1.2,44.33,3.34,1688.92,0.372,0.449333,0.539592,0.449333
71,1.2,57.33,4.17,1996.29,0.446,0.449333,0.539592,0.449333
72,1.2,70.00,5.01,2177.65,0.488,0.449333,0.539592,0.449333
73,1.2,80.17,5.83,2528.29,0.470,0.449333,0.539592,0.449333


In [37]:
df.to_csv('ME_pred.csv')

### Conclusion
- GP algorithm provides more accurate results than XGB and ANN.